<h1>QIIME2 Workflow</h1>

This notebook is a guide on working with QIIME2 with raw paired-end demultiplexed reads as the starting dataset. This notebook includes quality checking of raw reads, primer trimming, OTU picking, taxonomic assignment, and exporting data.

This workflow was built with the following as the main references: <a href = 'https://github.com/LangilleLab/microbiome_helper/wiki/Amplicon-SOP-v2-(qiime2-2020.8)'>LangilleLab SOP</a>, <a href = 'https://docs.qiime2.org/2021.2/tutorials/moving-pictures/'>"Moving pictures" Tutorial</a>, and <a href = 'https://docs.qiime2.org/2021.2/tutorials/atacama-soils/'>"Atacama soil microbiome" tutorial</a>.

Written for Day 1 of Bioinformatics Workshop by the Microbial Oceanography Laboratory. Credits: LBR dela Peña, BW Hingpit, JB Quijano, D Purganan. 

<span style='color:red'>**Checkpoint:**</span> Import QIIME 2 python module to allow for inline visualizations

In [ ]:
import qiime2 as q2

### Making the manifest file

Before we import our data, we have to make a **manifest file** that contains links to the forward and reverse file paths of each sample.

In [ ]:
import pandas as pd
import glob
import os

sampleIDs, forwardpaths, reversepaths = [],[],[]
fpath = os.getcwd()+"/raw-sequences/"
for filepath in (glob.glob(fpath+"*.gz")):
    sample = filepath.split("/")[-1].split("_")[0]
    if sample not in sampleIDs:
        sampleIDs.append(sample)
    if "R1_001.fastq.gz" in filepath:
        forwardpaths.append(filepath)
    elif "R2_001.fastq.gz" in filepath:
        reversepaths.append(filepath)

manifest =  pd.DataFrame({'sampleID': sorted(sampleIDs), 'forward-absolute-filepath': sorted(forwardpaths), 'reverse-absolute-filepath':sorted(reversepaths)} ) 
with open('manifest.txt', 'w') as m:
    print(manifest.to_csv(sep='\t', index=False, header=True), file=m)

This [manifest file](data/manifest.txt) will show the sample ID and the absolute paths to the forward and reverse reads.

### Importing the sequences
Now that we prepared all the necessary files, we can make our first QIIME command: importing the sequence data.

In [ ]:
# Import the sequences
# Insert path to sequence folder after '--input-path'
!qiime tools import \
    --type 'SampleData[PairedEndSequencesWithQuality]' \
    --input-path manifest.txt \
    --output-path raw-sequences/seqs.qza \
    --input-format PairedEndFastqManifestPhred33V2

This converts the sequence data into a **QIIME artifact**. Artifacts have the extension '.qza'

### Quality checking
Our sequences are already *demultiplexed*, meaning they are already separated into different samples. We can use the `demux` plugin instead to visualize our sequences. **QIIME visualizations** have the extension '.qzv'. The .qzv files can be viewed in  http://view.qiime2.org or we can import the `qiime2` module to view the visualizations inline.

In [ ]:
# Make summary of the QIIME2 artifact (.qza file)
!qiime demux summarize \
    --i-data  raw-sequences/seqs.qza \
    --o-visualization raw-sequences/seqs.qzv

# Visualize
q2.Visualization.load('raw-sequences/seqs.qzv')

Open the visualization summary and go to the **Interactive Quality Plot**. Here, we can see the average quality score of the reads at each position. In general, we want to maintain a score above 30. 

## Data Processing
To prepare our sequences, we have to perform several steps:
1. Trim primers
2. DADA2:
- Merge paired-end reads
- Filter sequences by quality
- Dereplicate

### Trim primers
To remove the primers in our sequences, we use the `cutadapt` plugin. The primers used were E572F/E1009R, which have **18bp** and **20bp** lengths, respectively. Removing the primers is important especially if there are ambiguous bases, which might get confused as chimeric or low quality positions. You can explore more about the primer sequences, length, and predicted amplicon size in this excellent app [PR-2 Primers](https://app.pr2-primers.org/). 

In [ ]:
# Make directory
%mkdir cleanup \

!qiime cutadapt trim-paired \
    --i-demultiplexed-sequences raw-sequences/seqs.qza \
    --p-front-f CYGCGGTAATTCCAGCTC \
    --p-front-r AYGGTATCTRATCRTCTTYG \
    --p-error-rate 0 \
    --o-trimmed-sequences cleanup/primer-trimmed-seqs.qza

In [ ]:
# Make summary of the QIIME2 artifact (.qza file)
# To check whether the primers were removed
!qiime demux summarize \
    --i-data  cleanup/primer-trimmed-seqs.qza \
    --o-visualization cleanup/primer-trimmed-seqs.qzv \

# Visualize
q2.Visualization.load('cleanup/primer-trimmed-seqs.qzv')

In [ ]:
q2.Visualization.load('cleanup/primer-trimmed-seqs.qzv')

<visualization: Visualization uuid: 14bb287e-f284-4d16-8146-ad4d97be9622>

### DADA2
DADA2 is a pipeline for detecting and correcting (where possible) Illumina amplicon sequence data. As implemented in the q2-dada2 plugin, this quality control process will additionally filter any phiX reads (commonly present in marker gene Illumina sequence data) that are identified in the sequencing data, and will filter chimeric sequences. DADA2 includes quality filtering, dereplicating, chimera filtering and singleton filtering.

<span style='color:red'>**Note:**</span> Several trimming and truncating positions will be done to check the number of sequences that will pass DADA2 filter

In DADA2, paired-end sequences are recommended to have at least 12-20 nt overlap. Considering that the 18S rRNA gene has an average length of 350-450 bases, trimming and truncation of paired reads should allow the recommended overlap.

<span style='color:blue'>Un-trimmed and un-truncated reads</span>

In [ ]:
# Un-trimmed and un-truncated
!qiime dada2 denoise-paired \
    --i-demultiplexed-seqs cleanup/primer-trimmed-seqs.qza \
    --p-trunc-len-f 0 \
    --p-trunc-len-r 0 \
    --o-table cleanup/table-all.qza \
    --o-representative-sequences cleanup/rep-seqs-all.qza \
    --o-denoising-stats cleanup/denoising-stats-all.qza

In [ ]:
# View statistics
!qiime metadata tabulate \
    --m-input-file cleanup/denoising-stats-all.qza \
    --o-visualization cleanup/denoising-stats-all.qzv \

q2.Visualization.load('cleanup/denoising-stats-all.qzv')

<span style='color:blue'>Truncated at the 250th nucleotide</span>

In [ ]:
# Truncated at 250th nucleotide
!qiime dada2 denoise-paired \
    --i-demultiplexed-seqs cleanup/primer-trimmed-seqs.qza \
    --p-trunc-len-f 250 \
    --p-trunc-len-r 250 \
    --o-table cleanup/table-250.qza \
    --o-representative-sequences cleanup/rep-seqs-250.qza \
    --o-denoising-stats cleanup/denoising-stats-250.qza

In [ ]:
# View statistics
!qiime metadata tabulate \
    --m-input-file cleanup/denoising-stats-250.qza \
    --o-visualization cleanup/denoising-stats-250.qzv \

q2.Visualization.load('cleanup/denoising-stats-250.qzv')

In [ ]:
q2.Visualization.load('cleanup/denoising-stats-250.qzv')

<visualization: Visualization uuid: acdcf06b-b9ba-4fa9-980c-70e670a6bfe5>

## Assigning taxonomy
After quality filtering, the resulting data can be explored using `feature-table summarize` and `feature table tabulate-seqs`. The former command will give information on how many sequences are associated with each sample and with each feature (ASVs), histograms of those and some related summary statistics while the latter will provide a mapping of feature IDs to sequences, and provide links to easily BLAST each sequence against the NCBI nt database.

In [ ]:
# Summarize ASV table (un-truncated/un-trimmed)
!qiime feature-table summarize \
    --i-table cleanup/table-all.qza \
    --o-visualization cleanup/table-all.qzv \
    --m-sample-metadata-file metadata.tsv

# Visualize
q2.Visualization.load('cleanup/table-all.qzv')

In [ ]:
# Summarize ASV table (truncated at 250th nucleotide)
!qiime feature-table summarize \
    --i-table cleanup/table-250.qza \
    --o-visualization cleanup/table-250.qzv \
    --m-sample-metadata-file metadata.tsv

# Visualize
q2.Visualization.load('cleanup/table-250.qzv')

In [ ]:
# Visualize
q2.Visualization.load('cleanup/table-250.qzv')

<visualization: Visualization uuid: dbd97667-ce5d-4409-b882-469ab8ff1d08>

The results show how many OTUs were determined in our samples in the overview. In the *Interactive Sample Detail* tab, you can see how many OTUs were detected in each sample. In the last tab (*Feature Detail*), you can see the OTU ID, their frequencies, and occurence in the samples. 
🧐 What is the most frequently detected OTU?

At this point, sequences truncated at the 250th nucleotide base will be used as more sequences were retrieved.

In [ ]:
# Map OTUs to sequences
!qiime feature-table tabulate-seqs \
    --i-data cleanup/rep-seqs-250.qza \
    --o-visualization cleanup/rep-seqs-250.qzv

# Visualize
q2.Visualization.load('cleanup/rep-seqs-250.qzv')

### Taxonomy assignment using SILVA

To annotate the metabarcoding data, we use a reference database which will classify the sequences to their taxonomic identities using the plugin `sci-kit learn`. For 18S rRNA eukaryotic data, we will use a classifier made from SILVA 138.1 optimized for the specific region targeted by the primers used in this run (18S V4 region). Other references for eukaryotic sequences can be used, such as the [PR2 database](https://pr2-database.org/) which has high-quality reference sequences curated by other experts.

In [ ]:
# Classify using sci-kit learn (sklearn)
!qiime feature-classifier classify-sklearn \
    --i-classifier ../refdb-2/silva-138.1-euk-v4-200700-classifier.qza \
    --i-reads cleanup/rep-seqs-250.qza \
    --o-classification cleanup/asv-taxa-250.qza

In [ ]:
# Visualize
!qiime metadata tabulate \
    --m-input-file cleanup/asv-taxa-250.qza \
    --o-visualization cleanup/asv-taxa-250.qzv

q2.Visualization.load('cleanup/asv-taxa-250.qzv')

We can view interactive taxonomic barplot to see the composition of each sample.

After loading the visualization, select *Level* to 7 or 8 (when Kingdom is included) to view at the most resolved classification.

In [ ]:
# sklearn
!qiime taxa barplot \
    --i-table cleanup/table-250.qza \
    --i-taxonomy cleanup/asv-taxa-250.qza \
    --m-metadata-file metadata.tsv \
    --o-visualization cleanup/bar-plots-asv-250.qzv

q2.Visualization.load('cleanup/bar-plots-asv-250.qzv')

## Filtering ASV table and representative sequences
The ASV table and representative sequences should be filtered to exclude putative metazoans (animal, multi-cellular chlorophytes and fungal sequencces). This filtering is done to focus on protists only.

In [ ]:
# Filtering ASV table
!qiime taxa filter-table \
    --i-table cleanup/table-250.qza \
    --i-taxonomy cleanup/asv-taxa-250.qza \
    --p-exclude k__Rhodophyceae,k__Fungi,k__Animalia,c__Embryophyta,c__Ulvophyceae \
    --o-filtered-table cleanup/asv-table-nometaz.qza \

# Filtering representative sequences
!qiime taxa filter-seqs \
    --i-sequences cleanup/rep-seqs-250.qza \
    --i-taxonomy cleanup/asv-taxa-250.qza \
    --p-exclude  k__Rhodophyceae,k__Fungi,k__Animalia,c__Embryophyta,c__Ulvophyceae \
    --o-filtered-sequences cleanup/asv-rep-seqs-nometaz-250.qza

Double check taxonomy barplots

In [ ]:
# sklearn
!qiime taxa barplot \
    --i-table cleanup/asv-table-nometaz.qza \
    --i-taxonomy cleanup/asv-taxa-250.qza \
    --m-metadata-file metadata.tsv \
    --o-visualization cleanup/bar-plots-asv-nometaz.qzv

In [ ]:
# Visualize
q2.Visualization.load('cleanup/bar-plots-asv-nometaz.qzv')

<visualization: Visualization uuid: a28df0a7-f5c9-4ff1-8222-6fec7fbdceaf>

Double check ASV table

In [ ]:
# Summarize ASV table (truncated at 250th nucleotide)
!qiime feature-table summarize \
    --i-table cleanup/asv-table-nometaz.qza \
    --o-visualization cleanup/asv-table-nometaz.qzv \
    --m-sample-metadata-file metadata.tsv

# Visualize
q2.Visualization.load('cleanup/asv-table-nometaz.qzv')

In [ ]:
# Visualize
q2.Visualization.load('cleanup/asv-table-nometaz.qzv')

<visualization: Visualization uuid: 5eab02f5-6f18-48bc-b574-ed8ea89bba74>

## Alpha Rarefaction Curves
### Checking the number of features per sample
In this step, the filtered ASV table will be used to view the alpha rarefaction curves.

Alpha rarefaction curves can be used to visualize whether a sample has been sufficiently sequenced to represent its true diversity.

To view the rarefaction curves for all the samples, check the number of features/OTUs per sample in the filtered ASV table visualization file. Run the code block below and go to the **Interactive Sample Detail** tab to check for the sample with the most numerous features/OTUs.

In [ ]:
q2.Visualization.load('cleanup/asv-table-nometaz.qzv')

<visualization: Visualization uuid: 5eab02f5-6f18-48bc-b574-ed8ea89bba74>

### Making alpha rarefaction curves
In the **Interactive Sample Detail**, take note of the highest ASV count. To view the rarefaction curves for all the samples, the highest number of ASV count will be set as the `--p-max-depth`.

In [ ]:
!qiime diversity alpha-rarefaction \
  --i-table cleanup/asv-table-nometaz.qza \
  --p-max-depth 2000 \
  --p-metrics 'shannon' \
  --m-metadata-file metadata.tsv \
  --o-visualization cleanup/asv-table-nometaz-arare.qzv \

# Visualize
q2.Visualization.load('cleanup/asv-table-nometaz-arare.qzv')

In [ ]:
# Visualize
q2.Visualization.load('cleanup/asv-table-nometaz-arare.qzv')

<visualization: Visualization uuid: ecd55c77-7753-42b6-b7eb-b0326e23f8a7>

### Choosing a sampling depth and making a phylogenetic tree
After seeing the rarefaction curves, we will select the sampling depth where we will rarefy the samples. Rarefaction will standardize the number of ASVs to the smallest number or ASVs in a sample which will allow for comparision between sites.

NOTE: The smallest number of ASVs in a sample is not always used as the sampling depth. This decision should be based on the rarefaction curves. If the selected sampling depth is not on the plateau-ed part of the curve, problems in statistical analyses may arise as the actual diversity for other samples may be reduced.

Before rarefying the samples and compute for alpha and beta diversity indices, run the code block below to make a tree file which will be used for phylogeny-based diversity analyses (e.g., Faith's PD and Unifrac).

In [ ]:
# Generate a tree for phylogenetic diversity analyses
%mkdir cleanup/phylogeny

!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences cleanup/asv-rep-seqs-nometaz-250.qza \
  --o-alignment cleanup/phylogeny/aligned-rep-seqs.qza \
  --o-masked-alignment cleanup/phylogeny/masked-aligned-rep-seqs.qza \
  --o-tree cleanup/phylogeny/unrooted-tree.qza \
  --o-rooted-tree cleanup/phylogeny/rooted-tree.qza

### Rarefying samples
As the rarefaction curves for the samples seem to have plateau-ed in the smallest number of ASVs (i.e., 927), sampling depth will be set to this number.

In [ ]:
# Alpha and Beta Diversity Analyses
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny cleanup/phylogeny/rooted-tree.qza \
  --i-table cleanup/asv-table-nometaz.qza \
  --p-sampling-depth 801 \
  --m-metadata-file metadata.tsv \
  --output-dir cleanup/core-metrics-results

In [ ]:
# Summarize ASV table (rarefied)
!qiime feature-table summarize \
    --i-table cleanup/core-metrics-results/rarefied_table.qza \
    --o-visualization cleanup/rarefied-table.qzv \
    --m-sample-metadata-file metadata.tsv

In [ ]:
# Visualize
q2.Visualization.load('cleanup/rarefied-table.qzv')

<visualization: Visualization uuid: 804f4828-b69c-4bcb-82b6-123547b4a4c3>

## PCoA plots and statistical testing
### Visualizing beta diversity indices
After looking for the discontinuity of data using hierarchical clustering, grouping of sites will then be viewed in a multidimensional space.

Run the code blocks below to view the PCoA plot of each beta diversity metric

In [ ]:
# Jaccard distance
q2.Visualization.load('cleanup/core-metrics-results/jaccard_emperor.qzv')

In [ ]:
# Bray-Curtis dissimilarity
q2.Visualization.load('cleanup/core-metrics-results/bray_curtis_emperor.qzv')

In [ ]:
# Unweighted Unifrac
q2.Visualization.load('cleanup/core-metrics-results/unweighted_unifrac_emperor.qzv')

In [ ]:
# Weighted Unifrac
q2.Visualization.load('cleanup/core-metrics-results/weighted_unifrac_emperor.qzv')

<visualization: Visualization uuid: 286834cf-9a42-4d5a-8aef-d9d18c4bfae4>

### Alpha significance testing between/among groups
After visualizing in a multidimensional space and deciding on clusters, significance testing of the groups based on their alpha diversity will be done. This is explored to check if a cluster of sites is significantly more diverse than the other.

Run the code blocks below to test for significance and view the results.

In [ ]:
# Make a directory for significance testing
%mkdir sig-test

# Shannon
!qiime diversity alpha-group-significance \
  --i-alpha-diversity cleanup/core-metrics-results/shannon_vector.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization sig-test/shannon-group-significance.qzv

q2.Visualization.load('sig-test/shannon-group-significance.qzv')

In [ ]:
# Faith's PD
!qiime diversity alpha-group-significance \
  --i-alpha-diversity cleanup/core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization sig-test/faith_pd-group-significance.qzv

q2.Visualization.load('sig-test/faith_pd-group-significance.qzv')

In [ ]:
q2.Visualization.load('sig-test/faith_pd-group-significance.qzv')

<visualization: Visualization uuid: 616317d4-d2bf-47aa-a709-262387b179c4>

In [ ]:
# Observed ASVs
!qiime diversity alpha-group-significance \
  --i-alpha-diversity cleanup/core-metrics-results/observed_features_vector.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization sig-test/observed_features-group-significance.qzv

q2.Visualization.load('sig-test/observed_features-group-significance.qzv')

In [ ]:
q2.Visualization.load('sig-test/observed_features-group-significance.qzv')

<visualization: Visualization uuid: 39bbb358-e43d-4796-8a25-a428364864a9>

In [ ]:
# Evenness
!qiime diversity alpha-group-significance \
  --i-alpha-diversity cleanup/core-metrics-results/evenness_vector.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization sig-test/evenness_vector-group-significance.qzv

q2.Visualization.load('sig-test/evenness_vector-group-significance.qzv')

In [ ]:
q2.Visualization.load('sig-test/evenness_vector-group-significance.qzv')

<visualization: Visualization uuid: f944adcd-0687-4095-90f2-4113cd917c6d>

### Beta significance testing between/among groups
Then, significance testing of the groups based on their beta diversity will be examined. This is done to check if there is a significant difference in the composition of communities between/among groups.

Run the code blocks below to test for significance and view the results.

In [ ]:
# Bray-Curtis
# Plastics-attached vs free-living
!qiime diversity beta-group-significance \
  --i-distance-matrix cleanup/core-metrics-results/bray_curtis_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column environment1 \
  --o-visualization sig-test/bray-curtis-environment1-significance.qzv \
  --p-pairwise

q2.Visualization.load('sig-test/bray-curtis-environment1-significance.qzv')

In [ ]:
# Weighted Unifrac
# Plastics-attached vs free-living
!qiime diversity beta-group-significance \
  --i-distance-matrix cleanup/core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column environment1 \
  --o-visualization sig-test/unweighted-unifrac-environment1-significance.qzv \
  --p-pairwise

q2.Visualization.load('sig-test/unweighted-unifrac-environment1-significance.qzv')

Saved Visualization to: sig-test/unweighted-unifrac-environment1-significance.qzv


<visualization: Visualization uuid: 10891af8-632b-4a8e-b3ab-9e55858e3dbf>

### Exporting ASV tables

We can export the ASV tables in a format we can use in other programs, such as R.

In [ ]:
# NOT RARIFIED
!qiime tools export --input-path cleanup/asv-table-nometaz.qza  --output-path exported-unrare \

!biom convert\
    -i exported-unrare/feature-table.biom\
    -o exported-unrare/feature-table-unrare.tsv\
    --to-tsv

In [ ]:
# RAREFIED
!qiime tools export --input-path cleanup/core-metrics-results/rarefied_table.qza  --output-path exported
!qiime tools export --input-path cleanup/asv-taxa-250.qza --output-path exported

# Change the first line of biom-taxonomy.tsv (i.e. the header) to this:
# #ASVID taxonomy confidence
!sed '1c#ASVID\ttaxonomy\tconfidence' exported/taxonomy.tsv > exported/biom-taxonomy.tsv

In [ ]:
!biom add-metadata \
    -i exported/feature-table.biom \
    -o exported/table-with-taxonomy.biom \
    --observation-metadata-fp exported/biom-taxonomy.tsv \
    --sc-separated taxonomy

!biom convert\
    -i exported/table-with-taxonomy.biom\
    -o exported/feature-table-with-tax.tsv\
    --to-tsv \
    --header-key taxonomy

In [ ]:
# Genus level
!qiime taxa collapse \
    --i-table cleanup/core-metrics-results/rarefied_table.qza \
    --i-taxonomy cleanup/asv-taxa-250.qza \
    --p-level 7 \
    --o-collapsed-table cleanup/table-genus.qza

!qiime tools export \
    --input-path cleanup/table-genus.qza \
    --output-path exported-genus

!biom convert\
    -i exported-genus/feature-table.biom\
    -o exported-genus/feature-table-with-tax.tsv\
    --to-tsv \
    --header-key taxonomy

Saved FeatureTable[Frequency] to: cleanup/table-genus.qza
Exported cleanup/table-genus.qza as BIOMV210DirFmt to directory exported-genus


In [ ]:
# Class level
!qiime taxa collapse \
    --i-table cleanup/core-metrics-results/rarefied_table.qza \
    --i-taxonomy cleanup/asv-taxa-250.qza \
    --p-level 4 \
    --o-collapsed-table cleanup/table-class.qza

!qiime tools export \
    --input-path cleanup/table-class.qza \
    --output-path exported

!biom convert\
    -i exported/table-with-taxonomy.biom\
    -o exported/feature-table-with-tax.tsv\
    --to-tsv \
    --header-key taxonomy

In [ ]:
# Kingdom level
!qiime taxa collapse \
    --i-table cleanup/core-metrics-results/rarefied_table.qza \
    --i-taxonomy cleanup/asv-taxa-250.qza \
    --p-level 2 \
    --o-collapsed-table cleanup/table-kingdom.qza

!qiime tools export \
    --input-path cleanup/table-kingdom.qza \
    --output-path exported-kingdom

!biom convert\
    -i exported-kingdom/feature-table.biom\
    -o exported-kingdom/feature-table-with-tax.tsv\
    --to-tsv \
    --header-key taxonomy

Saved FeatureTable[Frequency] to: cleanup/table-kingdom.qza
Exported cleanup/table-kingdom.qza as BIOMV210DirFmt to directory exported-kingdom


In [ ]:
# Phylum level
!qiime taxa collapse \
    --i-table cleanup/core-metrics-results/rarefied_table.qza \
    --i-taxonomy cleanup/asv-taxa-250.qza \
    --p-level 3 \
    --o-collapsed-table cleanup/table-phylum.qza

!qiime tools export \
    --input-path cleanup/table-phylum.qza \
    --output-path exported-phylum

!biom convert\
    -i exported-phylum/feature-table.biom\
    -o exported-phylum/feature-table-with-tax.tsv\
    --to-tsv \
    --header-key taxonomy

## ANCOM

In [ ]:
# Phylum level
!qiime composition add-pseudocount \
    --i-table cleanup/table-phylum.qza \
    --o-composition-table cleanup/comp-table.qza

Saved FeatureTable[Composition] to: cleanup/comp-table.qza


In [ ]:
!qiime composition ancom \
    --i-table cleanup/comp-table.qza\
    --m-metadata-file metadata.tsv \
    --m-metadata-column environment1 \
    --o-visualization cleanup/comp-ancom.qzv \

q2.Visualization.load('cleanup/comp-ancom.qzv')

Saved Visualization to: cleanup/comp-ancom.qzv


<visualization: Visualization uuid: ae745913-09ad-4120-9270-b87a53df97aa>

In [ ]:
q2.Visualization.load('cleanup/comp-ancom.qzv')

<visualization: Visualization uuid: 85edf951-0d7c-475c-9719-bee9f9f2fe3b>